In [ ]:
from sqlalchemy.engine import Engine, Connection
from sqlalchemy import create_engine
from os import getenv
from typing import List, Dict, Optional
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import matplotlib.pyplot as plt
from datetime import datetime
import datetime as dt
import numpy as np
from numpy import ndarray
import pytz

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
try:
    from dotenv import load_dotenv

    load_dotenv()
except:
    print('No ".env" file or python-dotenv not installed... Using default env variables...')

In [ ]:
def db_connection() -> Engine:
    dbname: Optional[str] = getenv('POSTGRES_DB_NAME')
    host: Optional[str] = getenv('POSTGRES_HOST')
    user: Optional[str] = getenv('POSTGRES_USERNAME')
    password: Optional[str] = getenv('POSTGRES_PASSWORD')
    port: Optional[str] = getenv('POSTGRES_PORT')
        
    postgres_str: str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    
    engine: Engine = create_engine(postgres_str)
    
    return engine

# WHAT WE HAVE TO QUERY

let's start by define date intervals

In [ ]:
startDate: str = (datetime.now(pytz.timezone('UTC')) - dt.timedelta(hours=167))
endDate: str = datetime.now(pytz.timezone('UTC'))
startDate, endDate 

In [ ]:
targetDate: datetime = endDate + dt.timedelta(hours=12)
targetDates: List[datetime] = [endDate + dt.timedelta(hours=i) for i in range(1,13)]

CONVERT INTO STRINGS AND PUSHED TO THE START OF THE HOUR (xx:00:00)

In [ ]:
start: str = startDate.strftime("%Y-%m-%d %H:00:00")
end: str = endDate.strftime("%Y-%m-%d %H:00:00")
targets: List[str] = [date.strftime("%Y-%m-%d %H:00:00") for date in targetDates]

In [ ]:
start, end

In [ ]:
targets

# QUERY OBSERVED INPUTS

where are they?

table: "meteomatics_weather" (for start to end - 1)

table: "meteomatics_forecast_weather" (for end)

In [ ]:
query1: str = "SELECT * FROM meteomatics_weather WHERE timestamp_utc between '{}' and '{}'"
observed_df1: DataFrame = pd.read_sql_query(query1.format(start, end), con=db_connection())
observed_df1.drop(['id'], axis=1, inplace=True)
observed_df1.rename(columns={'timestamp_utc': 'time'}, inplace=True)

In [ ]:
observed_df1

In [ ]:
query2: str = "SELECT * FROM meteomatics_forecast_weather WHERE forecast_timestamp_utc = '{}' and timestamp_query_utc = '{}'"
observed_df2: DataFrame = pd.read_sql_query(query2.format(end, end), con=db_connection())
observed_df2.drop(['id','timestamp_query_utc'], axis=1, inplace=True)
observed_df2.rename(columns={'forecast_timestamp_utc': 'time'}, inplace=True)

In [ ]:
observed_df2

In [ ]:
observed_df2.dtypes

CONCAT THE TWO OBSERVED DF

In [ ]:
farm_list: List = ['UP_PRCLCDPLRM_1',
'UP_PRCLCDMZRD_1',
'UP_PRCLCDPRZZ_1',
'UP_PRCLCMINEO_1',
'UP_PEPIZZA_1',
'UP_MPNTLCSMBC_1',
'UP_MPNTLCDMRN_1']

In [ ]:
observed_df: DataFrame = pd.concat([observed_df1, observed_df2], axis=0, ignore_index=True)
observed_df: DataFrame = observed_df.sort_values(by=['time','plant_code'], ascending=True, ignore_index=True)
observed_df: DataFrame = observed_df[observed_df['plant_code'].isin(farm_list)]

In [ ]:
observed_df

# QUERY KNOWN INPUTS (FORECASTS)

where are they?

table: "meteomatics_forecast_weather" 

In [ ]:
# query_fore: str = "SELECT * FROM meteomatics_forecast_weather WHERE forecast_timestamp_utc between '{}' and '{}'"
# known_df: DataFrame = pd.read_sql_query(query_fore.format(targets[0], targets[-1]), con=db_connection())
# known_df.drop(['id', 'timestamp_query_utc'], axis=1, inplace=True)
# # known_df.drop(['id'], axis=1, inplace=True)
# known_df: DataFrame = known_df.sort_values(by=['forecast_timestamp_utc','plant_code'], ascending=True, ignore_index=True)
# known_df: DataFrame = known_df[known_df['plant_code'].isin(farm_list)]

In [ ]:
query_fore: str = "SELECT * FROM meteomatics_forecast_weather WHERE forecast_timestamp_utc between '{}' and '{}'"
known_df: DataFrame = pd.read_sql_query(query_fore.format(targets[0], targets[-1]), con=db_connection())
known_df.drop(['id'], axis=1, inplace=True)
known_df: DataFrame = known_df.sort_values(by=['forecast_timestamp_utc', 'plant_code'], ascending=True, ignore_index=True)
known_df['diff'] = known_df['forecast_timestamp_utc'] - known_df['timestamp_query_utc']
known_df = known_df.sort_values('diff', ascending=True).drop_duplicates(subset=['plant_code', 'forecast_timestamp_utc'], keep='first')
# assert known_df['timestamp_query_utc'].unique() == pd.Timestamp(targets[0])
known_df.drop(['timestamp_query_utc'], axis=1, inplace=True)
known_df: DataFrame = known_df[known_df['plant_code'].isin(farm_list)]
known_df.rename(columns={'forecast_timestamp_utc': 'time'}, inplace=True)

# TEST observed_df

In [ ]:
maserio_ob = observed_df[observed_df['plant_code']=='UP_MPNTLCDMRN_1']

In [ ]:
assert len(maserio_ob) == 168

In [ ]:
maserio_kn = known_df[known_df['plant_code']=='UP_MPNTLCDMRN_1']

In [ ]:
maserio_kn

# CONCATENATE OBSERVED AND KNOWN

In [ ]:
columns = ['plant_code', 'time', 'dew_point_2m_C', 'temperature_2m_C', 'msl_pressure_hPa', 'sfc_pressure_hPa',
           'precipitation_1h_mm', 'wind_speed_mean_10m_1h_ms',
           'wind_speed_mean_100m_1h_ms', 'wind_dir_mean_100m_1h_d', 'wind_dir_mean_10m_1h_d', 'wind_gusts_10m_1h_ms',
           'wind_gusts_10m_ms']

In [ ]:
observed_df = observed_df[columns]
known_df = known_df[columns]

In [ ]:
df: DataFrame = pd.concat([observed_df, known_df], axis=0, ignore_index=True)
df = df.sort_values(['plant_code', 'time'], ascending=True, ignore_index=True)

In [ ]:
# add other engineered features
timestamp_s: Series = df['time'].map(datetime.timestamp)

day: int = 24 * 60 * 60
year: float = 365.2425 * day

df['Day sin']: Series = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos']: Series = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin']: Series = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos']: Series = np.cos(timestamp_s * (2 * np.pi / year))
    
earliest_time: Timestamp = df.time.min()
df['t']: Series = (df['time'] - earliest_time).dt.seconds / 60 / 60 + (df['time'] - earliest_time).dt.days * 24
df['days_from_start']: Series = (df['time'] - earliest_time).dt.days
df["id"] = df["plant_code"]
df['hour']: Series = df["time"].dt.hour
df['day']: Series = df["time"].dt.day
df['day_of_week']: Series = df["time"].dt.dayofweek
df['month']: Series = df["time"].dt.month
df['categorical_id']: Series = df['id'].copy()
df['hours_from_start']: Series = df['t']
df['categorical_day_of_week']: Series = df['day_of_week'].copy()
df['categorical_hour']: Series = df['hour'].copy()

In [ ]:
df.head()